# Coleta

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import spacy
import os
from sklearn.feature_extraction.text import TfidfVectorizer

 # Configuração de Processamento

In [2]:
nltk.download("stopwords")
stopwords_pt = set(stopwords.words("portuguese"))

nlp = spacy.load("pt_core_news_sm")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\femor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
urls = [
    "https://www.infomoney.com.br/economia/previa-da-carga-tributaria-sobe-para-3232-do-pib-em-2024/",
    "https://www.seudinheiro.com/2025/bolsa-dolar/brava-energia-dispara-19-enquanto-embraer-desaba-11-veja-as-principais-altas-e-baixas-da-semana-na-bolsa/?xpromo=XD-ME-SD-SITE-MNT-X-X-X-X-X&utm_source=SD&utm_medium=MT&utm_campaign=XV-MI-SD-SITE-MNT-X-X-X-X-X&_gl=1*1lgx2ui*_ga*MjIxMTQ4NjgwLjE3NDMyODIwOTE.*_ga_BT4VXC4PQ6*MTc0MzI4MjA5MC4xLjAuMTc0MzI4MjA5MC42MC4wLjA."
]

os.makedirs("noticias", exist_ok=True)

# Funções de Coleta e Limpeza de Texto

In [ ]:
def coletar_texto(url):
    #Faz o scraping do site e retorna o texto principal da noticia.
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    paragrafos = soup.find_all("p")

    paragrafos_filtrados = [
        p.get_text().strip() for p in paragrafos
        if p.get_text().strip() and not any(kw in p.get_text().lower() for kw in ["publicidade", "compartilhe", "atualizado", "agência brasil"])
    ]

    return " ".join(paragrafos_filtrados)

In [5]:
def limpar_texto(texto):
    texto = texto.lower()  
    texto = re.sub(r"http\S+|www\S+", "", texto)  
    texto = re.sub(r"\d+", "", texto)  
    texto = re.sub(r"[^\w\s]", "", texto)  
    
    doc = nlp(texto)
    palavras = [token.lemma_ for token in doc if token.text not in stopwords_pt and not token.is_punct]

    return " ".join(palavras)

# Coleta e Salvamento das Notícias

In [6]:
for i, url in enumerate(urls):
    try:
        texto = coletar_texto(url)
        texto_limpo = limpar_texto(texto)

        # DataFrame individual
        df_noticia = pd.DataFrame([{
            "url": url,
            "texto_original": texto,
            "texto_limpo": texto_limpo,
            "num_palavras": len(texto.split())
        }])

       
        nome_arquivo = f"noticias/noticia_{i+1}.csv"

        df_noticia.to_csv(nome_arquivo, index=False)

        print(f"Notícia salva em: {nome_arquivo}")

        time.sleep(2)

    except Exception as e:
        print(f"Erro ao coletar {url}: {e}")

print("Processamento finalizado!")

Notícia salva em: noticias/noticia_1.csv
Notícia salva em: noticias/noticia_2.csv
Processamento finalizado!


In [7]:
for i, url in enumerate(urls):
    try:
        texto = coletar_texto(url)
        texto_limpo = limpar_texto(texto)

        # DataFrame individual
        df_noticia = pd.DataFrame([{
            "url": url,
            "texto_original": texto,
            "texto_limpo": texto_limpo,
            "num_palavras": len(texto.split())
        }])

       
        nome_arquivo = f"noticias/noticia_{i+1}.csv"

        df_noticia.to_csv(nome_arquivo, index=False)

        print(f"Notícia salva em: {nome_arquivo}")

        time.sleep(2)

    except Exception as e:
        print(f"Erro ao coletar {url}: {e}")

print("Processamento finalizado!")

Notícia salva em: noticias/noticia_1.csv
Notícia salva em: noticias/noticia_2.csv
Processamento finalizado!


In [8]:
arquivos = os.listdir("noticias")
dfs = [pd.read_csv(f"noticias/{arquivo}") for arquivo in arquivos if arquivo.endswith(".csv")]

df = pd.concat(dfs, ignore_index=True)

In [9]:
print("\nEstatísticas do Dataset:")
display(df.describe())


Estatísticas do Dataset:


,num_palavras
count,2.000000
mean,987.000000
std,519.016377
min,620.000000
25%,803.500000
50%,987.000000
75%,1170.500000
max,1354.000000


# Extração de Palavras-chave

In [10]:
df["num_palavras"] = df["texto_original"].apply(lambda x: len(str(x).split()))

print("\n Estatísticas de palavras por notícia:")
display(df["num_palavras"].describe())


 Estatísticas de palavras por notícia:


count       2.000000
mean      987.000000
std       519.016377
min       620.000000
25%       803.500000
50%       987.000000
75%      1170.500000
max      1354.000000
Name: num_palavras, dtype: float64

In [11]:
df["texto_limpo"] = df["texto_limpo"].fillna("")  
vectorizer = TfidfVectorizer(max_features=10)  
tfidf_matrix = vectorizer.fit_transform(df["texto_limpo"]) 

termos = vectorizer.get_feature_names_out()

df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=termos)

display(df_tfidf)

,alta,imposto,lucro,percentual,pib,poder,ponto,renda,sobre,subir
0,0.099170,0.322303,0.074378,0.383296,0.592366,0.000000,0.421474,0.074378,0.371888,0.247926
1,0.298715,0.049786,0.298715,0.000000,0.000000,0.839668,0.099572,0.248929,0.199144,0.049786
